<a href="https://colab.research.google.com/github/tg-bomze/Video2Anime/blob/master/AnimeGan_Video_(Rus).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Конвертер видео в Аниме</font></b>

<b><font color="black" size="+2">Базируется на:</font></b>

**GitHub репозиторий**: [AnimeGAN](https://github.com/TachibanaYoshino/AnimeGAN)

Автор: **[Tachibana Yoshino](https://github.com/TachibanaYoshino).**

<b><font color="black" size="+2">Колаб собрал:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).



```
Далее тыкай на кнопки (куда указывает красная стрелка) в каждом блоке поочередно. После нажатия дождись окончания выполнения.
```


```
Если возникают какие-либо ошибки, уберите галочки на delete_log, сделайте скриншот и обратитесь к автору Колаба.
```





In [0]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Устанавливаем все необходимые компоненты</font></b>
!pip install youtube_dl
!pip install ffmpeg
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
import youtube_dl
import io
import base64
import os
import cv2
from google.colab import files
%matplotlib inline

%tensorflow_version 1.x
!git clone  https://github.com/TachibanaYoshino/AnimeGAN.git
%cd AnimeGAN
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/vgg16%2F19.npy/vgg19.npy
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/Haoyao-style_V1.0/Haoyao-style.zip
!unzip /content/AnimeGAN/Haoyao-style.zip -d pppoint
!rm /content/AnimeGAN/Haoyao-style.zip
!cp /content/AnimeGAN/vgg19.npy /content/AnimeGAN/vgg19_weight
!rm /content/AnimeGAN/vgg19.npy
!mkdir img

delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()

In [0]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Загрузите видео в формте mp4</font></b>

#@markdown *Ниже введите ссылку на видео (например, YouTube или Twitter), либо оставьте поле **source_url** пустым (в таком случае будет предложено загрузить ролик с компьютера).*

#@markdown *Тесты проводились на видеозаписях длинною до 1 минуты. Есть вероятность, что большие отрезки могут привести к ошибкам.*
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  file_name = "downloaded_video." + fn.split(".")[1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    file_name = "downloaded_video." + fn.split(".")[1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
!rm -rf img
!mkdir img
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))

delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()
#@markdown *При необходимости конвертировать видео в mp4 можно тут: https://convert-video-online.com*

#@markdown *Если при загрузки файла с компьютера возникнет ошибка, то запустите этот блок заново. Помимо этого не рекомендуется загружать видео, в названии которых есть пробелы.*

In [0]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Разбиваем видео на кадры</font></b>

vidcap = cv2.VideoCapture(file_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("img/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [0]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Запускаем конвертацию в аниме</font></b>

!python test.py --checkpoint_dir /content/AnimeGAN/pppoint --test_dir /content/AnimeGAN/img --style_name H
delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()

In [0]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Собираем кадры в видео</font></b>

frames = []
img = os.listdir("results/H/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/H/"+i))
frames = np.array(frames)
imageio.mimsave("final.mp4", frames, fps=fps_of_video)
!ffmpeg -i downloaded_video.mp4 -vn -ar 44100 -ac 2 -ab 192K -f mp3 sound.mp3
!ffmpeg -i sound.mp3 -i final.mp4 final_audio.mp4

print('Сборка завершена')
delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()

In [0]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Скачать итоговое видео</font></b>
files.download("final_audio.mp4")